## Table of Contents
- [Introduction](#introduction)
- [Data Wrangling](#wrangling)
    - [Gather](#gather)
    - [Assess](#assess)
    - [Clean](#clean)
    - [Analyze](#analyze)
    - [Visualize](#visualize)
- [Conclusions](#conclusions)

In [4]:
PATH = './data/'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

## I) Introduction <a id = "introduction">

**Aim:** Analyze absolute difference and (possibly) margin of error between stock market forecast of price returns and actual stock market price returns.

I will be analyzing quarterly price returns within the past 20 years for the firms present in the S&P 500 2019 Index.

> At first, I wanted to analyze the forecasted vs. actual price earnings of the S&P in its entirety for the past 20 years. However, considering that firms continuously enter and leave stock indices every year, there would be varying levels of inconsistencies and marginal errors when comparing annual S&P returns alone. To combat this problem, I have isolated these two approaches:
- Analyze the historical earnings of *only* the firms present in the S&P 2019 Index
- Keep track of all firms that were present in the S&P for the past 20 years. Keep track of how many times each firm appeared in the Index and for those with the least count, analyze them individually on how they differ from the firms that stayed for longer.


## II) Data Wrangling <a id="wrangling"></a>

To gather the data depicted under the `./data` folder, I used Bloomberg Excel functions.

### A) Gather <a id = "gather"></a>
> **APPROACH 1:** Focus on the firms that appear in the 2019 S&P Index and analyze their forecasted vs. actual price earnings for the last 20 years.

To ensure consistency in analysis among multiple firms, I divide both the forecasted and actual price earning dates by *calendar period* instead of fiscal period. This is because fiscal period differs by firm whereas calendar period is consistent by dates. 

**Historic *forecasted* and *actual* data from January 1999 until December 2019**

In [7]:
#historic forecasted EPS 
df_eps_fc = pd.read_csv(PATH + 'sp-eps-fc.csv')

#historic actual EPS
df_eps_act = pd.read_csv(PATH + 'sp-eps-act.csv')

#historic actual EPS
df_eod_act = pd.read_csv(PATH + 'sp-eod-act.csv')

#historic forecasted EPS with terms
df_eps_fc_terms = pd.read_csv(PATH + 'sp-eps-fc-terms.csv')

## B) Assess

### Forecasted Historical EPS



In [3]:
df_fc.sample(5)

,Forecast Made,Term Forecasted,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
16,10/1/2003,04Q1,NaN,NaN,0.423,0.008,NaN,NaN,-0.240,0.570,...,0.340,0.148,0.527,0.270,NaN,NaN,0.212,0.474,1.092,NaN
13,1/1/2003,03Q2,NaN,NaN,0.320,0.003,NaN,NaN,-0.375,0.539,...,0.300,0.107,0.493,0.260,NaN,NaN,0.242,0.389,0.984,NaN
49,1/1/2012,12Q2,0.791,-0.325,1.664,1.124,NaN,0.819,-0.091,1.238,...,0.340,0.437,2.050,0.589,1.121,0.493,0.701,1.331,0.375,NaN
58,4/1/2014,14Q3,0.785,1.630,1.863,1.241,0.785,0.926,0.140,0.618,...,0.747,0.545,1.864,0.608,1.118,0.543,1.101,1.385,0.470,0.368
5,1/1/2001,01Q2,NaN,NaN,NaN,0.006,NaN,NaN,-0.120,0.470,...,NaN,0.311,0.555,0.177,NaN,NaN,0.200,NaN,0.798,NaN


In [4]:
df_fc.shape

(80, 507)

**Observation:** There are 505 firms encompassing 80 quarterly forecast periods since 1999.

In [5]:
#number of rows with missing data
df_fc.isna().sum().max()

80

**Observation:** There are 80 rows with missing data, which means that all quarterly calendar periods contain incomplete data across all firms in the 2019 S&P Index.

In [6]:
#check for rows where all columns are NaN values.
columns_to_check = df_fc.columns[2:]
df_fc[columns_to_check].isnull().apply(lambda x: all(x), axis = 1).value_counts()

False    80
dtype: int64

In [7]:
#check for columns where all rows are NaN values.
df_eod[columns_to_check].isnull().apply(lambda x: all(x), axis = 0).value_counts()

False    505
dtype: int64

**Observations:**
- No quarterly calendar period is empty of data for all firms.
- No firm is empty of data for all calendar periods.

***This means that no singular calendar period has missing data for an entire firm, and
no singular firm has missing data for an entire calendar period.***

### Historic Stock Returns

In [8]:
#generate 10 random samples 
df_eod.sample(5)

,date,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
21,6/30/2004,19.7280,NaN,29.453,2.3243,NaN,14.5017,12.580,19.5026,27.48,...,NaN,33.3100,44.41,26.0500,38.2016,NaN,13.3816,88.20,61.4500,NaN
5,6/30/2000,49.6906,NaN,NaN,3.7411,NaN,7.5201,15.375,19.9435,NaN,...,NaN,82.5625,39.25,10.2708,54.6678,NaN,5.0783,NaN,45.8906,NaN
40,3/31/2009,10.9909,NaN,41.080,15.0171,NaN,16.3300,4.900,22.8232,27.49,...,NaN,19.1600,68.10,26.8500,11.9874,NaN,19.7596,36.50,9.8300,NaN
49,6/30/2011,36.5481,NaN,58.490,47.9529,NaN,41.4000,16.200,25.1773,60.42,...,NaN,36.4700,81.38,38.0800,27.4261,NaN,39.7206,63.20,24.0100,NaN
68,3/31/2016,39.8500,41.01,160.340,108.9900,57.12,86.5500,94.810,41.8300,115.40,...,NaN,47.4300,83.59,61.6300,29.4021,40.9,58.8546,106.63,24.2100,44.33


In [9]:
df_eod.shape

(84, 506)

**Observation:** There are 505 firms encompassing 84 calendar periods.

In [10]:
#number of rows with missing data
df_eod.isna().sum().max()

83

**Observation:** There are 83 rows with missing values, which means only one row has non-missing data.

In [11]:
#isolate row with no missing data
df_eod[~(df_eod.isna().any(axis=1))]

,date,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
82,9/30/2019,76.63,26.97,165.4,223.97,75.72,82.33,177.89,83.67,192.35,...,64.89,95.9,70.61,53.31,29.91,79.62,113.43,137.27,44.52,124.59


**Observation:** September 30, 2019 is the only recorded calendar time in the past 20 years that contains complete end-of-day stock prices for all firms in the 2019 S&P Index.

In [12]:
#check for rows where all columns are NaN values.
columns_to_check = df_eod.columns
df_eod[columns_to_check].isnull().apply(lambda x: all(x), axis = 1).value_counts()

False    84
dtype: int64

In [13]:
#check for columns where all rows are NaN values.
df_eod[columns_to_check].isnull().apply(lambda x: all(x), axis = 0).value_counts()

False    506
dtype: int64

**Observations:** 

- there is no calendar period that's empty of data for all firms.
- there is no firm that's empty of data for all calendar periods.

**Therefore, all calendar periods and firms have data for historical end of day stock price.**

In [14]:
#count how many rows have isolated data
df_eod.duplicated().sum()

0

**Observation:** There is no duplicated data among all firms for all calendar periods in `df_eod`.

In [15]:
#count number of repeated firm names
df_eod.columns.duplicated().sum()

0

**Observation:** There are no duplicated firms in `df_eod`.

### Quality

**Missing Data**

-  N/A
--- 

- firm names across both DataFrames are capitalized


### Tidiness

- both DataFrames need to be merged with firm names transposed into rows

## C) Cleaning

# III) Store Data

# IV) Explore Data

## Univariate

## Bivariate

## Multivariate

# V) Visualize Data